<a href="https://colab.research.google.com/github/Ddshsp/lab/blob/main/%D0%A8%D0%94%D0%94_%D0%9B%D0%A04_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from contextlib import contextmanager
from time import perf_counter
from typing import Callable

In [ ]:
@contextmanager
def measure_time() -> Callable[[], float]:
    begin = perf_counter()
    yield lambda: perf_counter() - begin
    print(f'Duration: {perf_counter() - begin:.3f} seconds')

In [ ]:
from time import perf_counter, sleep
from contextlib import contextmanager
from typing import Callable

@contextmanager
def time_tracker() -> Callable[[], float]:
    initial_time = perf_counter()
    yield lambda: perf_counter() - initial_time
    print(f'Elapsed time: {perf_counter() - initial_time:.3f} seconds')


In [ ]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!ls -la
!pwd

--2024-12-02 19:06:44--  https://downloads.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400864419 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.3-bin-hadoop3.tgz.2’

spark-3.5.3-bin-had 100%[===================>] 382.29M  12.8MB/s    in 32s     

2024-12-02 19:07:17 (11.9 MB/s) - ‘spark-3.5.3-bin-hadoop3.tgz.2’ saved [400864419/400864419]

total 1174440
drwxr-xr-x  1 root root      4096 Dec  2 19:06 .
drwxr-xr-x  1 root root      4096 Dec  2 18:32 ..
drwxr-xr-x  4 root root      4096 Nov 25 19:13 .config
drwxr-xr-x  5 root root      4096 Dec  2 19:00 output
drwxr-xr-x  1 root root      4096 Nov 25 19:13 sample_data
drwxr-xr-x 13 1000 1000      4096 Sep  9 05:33 spark-3.5.3-bin-hadoop3
-rw-r--r--  1 root root 4008

In [ ]:
!export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
!export SPARK_HOME=/content/spark-3.5.3-bin-hadoop3

In [ ]:
!pip install findspark
!echo $SPARK_HOME

/usr/local/lib/python3.10/dist-packages/pyspark


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
!ls /content/*

/content/spark-3.5.3-bin-hadoop3.tgz	/content/spark-3.5.3-bin-hadoop3.tgz.2
/content/spark-3.5.3-bin-hadoop3.tgz.1

/content/output:
data.csv  data.json  data.parquet

/content/sample_data:
anscombe.json		     california_housing_train.csv  mnist_train_small.csv
california_housing_test.csv  mnist_test.csv		   README.md

/content/spark-3.5.3-bin-hadoop3:
bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [ ]:
def func(n: int) -> pd.DataFrame:
    data = {
        "gender": np.random.choice(["Male", "Female"], n),
        "age": np.random.randint(18, 65, n),
        "income": np.random.randint(20000, 150000, n),
        "profession": np.random.choice(["Engineer", "Doctor", "Teacher", "Lawyer"], n),
    }
    return pd.DataFrame(data)

data = func(1_000_000)

In [ ]:
from pyspark.sql import Row

# Создание DataFrame из RDD
df = spark.createDataFrame(data)
rdd = df.rdd

# Создание Dataset (аналогично DataFrame, но ориентирован на типы)
# В PySpark нет отдельного типа Dataset, поэтому DataFrame используется в качестве Dataset
dataset = df


In [ ]:
!rm -rIf output/*

In [ ]:
# Запись DataFrame в CSV
%timeit df.write.csv("output/data.csv", header=True, mode="overwrite")

# Запись DataFrame в JSON
%timeit df.write.json("output/data.json", mode="overwrite")

# Запись DataFrame в PARQUET
%timeit df.write.parquet("output/data.parquet", mode="overwrite")


3.59 s ± 670 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.03 s ± 1.17 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.57 s ± 451 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Чтение RDD из CSV
%timeit rdd_from_csv = spark.sparkContext.textFile("output/data.csv")

# Чтение DataFrame из CSV
%timeit df_from_csv = spark.read.csv("output/data.csv", header=True, inferSchema=True)


25.5 ms ± 6.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.74 s ± 353 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Чтение RDD из JSON
%timeit rdd_from_json = spark.sparkContext.textFile("output/data.json")

# Чтение DataFrame из JSON
%timeit df_from_json = spark.read.json("output/data.json")


31.1 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.73 s ± 384 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Чтение DataFrame из PARQUET
%timeit df_from_parquet = spark.read.parquet("output/data.parquet")


91.8 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# Чтение всех CSV файлов
all_csv_rdd = spark.sparkContext.textFile("output/*.csv")
header = all_csv_rdd.first()
data_rdd = all_csv_rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
df_from_rdd = spark.createDataFrame(data_rdd, schema=header.split(","))

In [ ]:
df_from_rdd.take(10)

[Row(gender='Male', age='53', income='146783', profession='Teacher'),
 Row(gender='Male', age='38', income='102661', profession='Doctor'),
 Row(gender='Male', age='63', income='27239', profession='Engineer'),
 Row(gender='Male', age='49', income='133171', profession='Lawyer'),
 Row(gender='Female', age='33', income='144383', profession='Doctor'),
 Row(gender='Female', age='24', income='95867', profession='Engineer'),
 Row(gender='Male', age='52', income='112093', profession='Engineer'),
 Row(gender='Male', age='32', income='50041', profession='Doctor'),
 Row(gender='Female', age='45', income='26404', profession='Lawyer'),
 Row(gender='Male', age='55', income='42273', profession='Lawyer')]

In [ ]:
df_from_rdd.show(10)


+------+---+------+----------+
|gender|age|income|profession|
+------+---+------+----------+
|  Male| 53|146783|   Teacher|
|  Male| 38|102661|    Doctor|
|  Male| 63| 27239|  Engineer|
|  Male| 49|133171|    Lawyer|
|Female| 33|144383|    Doctor|
|Female| 24| 95867|  Engineer|
|  Male| 52|112093|  Engineer|
|  Male| 32| 50041|    Doctor|
|Female| 45| 26404|    Lawyer|
|  Male| 55| 42273|    Lawyer|
+------+---+------+----------+
only showing top 10 rows



In [ ]:
filtered_df = df_from_rdd.filter((df_from_rdd.age >= 25) & (df_from_rdd.age <= 50))
filtered_df.show()


+------+---+------+----------+
|gender|age|income|profession|
+------+---+------+----------+
|  Male| 38|102661|    Doctor|
|  Male| 49|133171|    Lawyer|
|Female| 33|144383|    Doctor|
|  Male| 32| 50041|    Doctor|
|Female| 45| 26404|    Lawyer|
|Female| 33|130449|    Lawyer|
|Female| 45| 29300|  Engineer|
|Female| 47| 76500|   Teacher|
|Female| 46| 81486|    Doctor|
|  Male| 31| 76713|    Lawyer|
|  Male| 41| 78577|  Engineer|
|  Male| 42| 23073|  Engineer|
|Female| 38|145958|  Engineer|
|Female| 44|149150|   Teacher|
|Female| 48| 71664|   Teacher|
|Female| 44| 94780|    Doctor|
|Female| 46| 20099|    Doctor|
|Female| 28|134153|  Engineer|
|  Male| 50| 49440|   Teacher|
|Female| 49| 91901|  Engineer|
+------+---+------+----------+
only showing top 20 rows



In [ ]:
df_from_rdd.select("age", "profession").show()


+---+----------+
|age|profession|
+---+----------+
| 53|   Teacher|
| 38|    Doctor|
| 63|  Engineer|
| 49|    Lawyer|
| 33|    Doctor|
| 24|  Engineer|
| 52|  Engineer|
| 32|    Doctor|
| 45|    Lawyer|
| 55|    Lawyer|
| 22|    Doctor|
| 33|    Lawyer|
| 45|  Engineer|
| 47|   Teacher|
| 18|   Teacher|
| 46|    Doctor|
| 31|    Lawyer|
| 56|    Doctor|
| 41|  Engineer|
| 42|  Engineer|
+---+----------+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F

df_from_rdd.select(F.avg("income"), F.max("age")).show()


+------------+--------+
| avg(income)|max(age)|
+------------+--------+
|84992.794062|      64|
+------------+--------+



In [ ]:
df_from_rdd.groupBy("gender", "profession").agg(F.max("income")).show()


+------+----------+-----------+
|gender|profession|max(income)|
+------+----------+-----------+
|Female|    Doctor|      99999|
|Female|  Engineer|      99999|
|Female|    Lawyer|      99999|
|Female|   Teacher|      99999|
|  Male|    Doctor|      99997|
|  Male|  Engineer|      99998|
|  Male|    Lawyer|      99999|
|  Male|   Teacher|      99999|
+------+----------+-----------+



In [ ]:
pivot_df = df_from_rdd.groupBy("gender", "profession").count().show()


+------+----------+------+
|gender|profession| count|
+------+----------+------+
|Female|    Doctor|125635|
|Female|   Teacher|125471|
|  Male|    Lawyer|125124|
|  Male|   Teacher|124900|
|Female|  Engineer|124658|
|Female|    Lawyer|124341|
|  Male|    Doctor|125525|
|  Male|  Engineer|124346|
+------+----------+------+



In [ ]:
df_with_ratio = df_from_rdd.withColumn("income_per_age", df_from_rdd.income / df_from_rdd.age)
df_with_ratio.show()


+------+---+------+----------+------------------+
|gender|age|income|profession|    income_per_age|
+------+---+------+----------+------------------+
|  Male| 53|146783|   Teacher| 2769.490566037736|
|  Male| 38|102661|    Doctor|2701.6052631578946|
|  Male| 63| 27239|  Engineer|432.36507936507934|
|  Male| 49|133171|    Lawyer|2717.7755102040815|
|Female| 33|144383|    Doctor| 4375.242424242424|
|Female| 24| 95867|  Engineer|3994.4583333333335|
|  Male| 52|112093|  Engineer|2155.6346153846152|
|  Male| 32| 50041|    Doctor|        1563.78125|
|Female| 45| 26404|    Lawyer| 586.7555555555556|
|  Male| 55| 42273|    Lawyer|             768.6|
|Female| 22| 37125|    Doctor|            1687.5|
|Female| 33|130449|    Lawyer|            3953.0|
|Female| 45| 29300|  Engineer| 651.1111111111111|
|Female| 47| 76500|   Teacher| 1627.659574468085|
|Female| 18|107528|   Teacher| 5973.777777777777|
|Female| 46| 81486|    Doctor|1771.4347826086957|
|  Male| 31| 76713|    Lawyer|2474.6129032258063|


In [ ]:
df_with_ratio = df_with_ratio.withColumnRenamed("income_per_age", "rub_per_year")
df_with_ratio.show()


+------+---+------+----------+------------------+
|gender|age|income|profession|      rub_per_year|
+------+---+------+----------+------------------+
|  Male| 53|146783|   Teacher| 2769.490566037736|
|  Male| 38|102661|    Doctor|2701.6052631578946|
|  Male| 63| 27239|  Engineer|432.36507936507934|
|  Male| 49|133171|    Lawyer|2717.7755102040815|
|Female| 33|144383|    Doctor| 4375.242424242424|
|Female| 24| 95867|  Engineer|3994.4583333333335|
|  Male| 52|112093|  Engineer|2155.6346153846152|
|  Male| 32| 50041|    Doctor|        1563.78125|
|Female| 45| 26404|    Lawyer| 586.7555555555556|
|  Male| 55| 42273|    Lawyer|             768.6|
|Female| 22| 37125|    Doctor|            1687.5|
|Female| 33|130449|    Lawyer|            3953.0|
|Female| 45| 29300|  Engineer| 651.1111111111111|
|Female| 47| 76500|   Teacher| 1627.659574468085|
|Female| 18|107528|   Teacher| 5973.777777777777|
|Female| 46| 81486|    Doctor|1771.4347826086957|
|  Male| 31| 76713|    Lawyer|2474.6129032258063|


In [ ]:
df_without_ratio = df_with_ratio.drop("rub_per_year")
df_without_ratio.show()


+------+---+------+----------+
|gender|age|income|profession|
+------+---+------+----------+
|  Male| 53|146783|   Teacher|
|  Male| 38|102661|    Doctor|
|  Male| 63| 27239|  Engineer|
|  Male| 49|133171|    Lawyer|
|Female| 33|144383|    Doctor|
|Female| 24| 95867|  Engineer|
|  Male| 52|112093|  Engineer|
|  Male| 32| 50041|    Doctor|
|Female| 45| 26404|    Lawyer|
|  Male| 55| 42273|    Lawyer|
|Female| 22| 37125|    Doctor|
|Female| 33|130449|    Lawyer|
|Female| 45| 29300|  Engineer|
|Female| 47| 76500|   Teacher|
|Female| 18|107528|   Teacher|
|Female| 46| 81486|    Doctor|
|  Male| 31| 76713|    Lawyer|
|  Male| 56| 95985|    Doctor|
|  Male| 41| 78577|  Engineer|
|  Male| 42| 23073|  Engineer|
+------+---+------+----------+
only showing top 20 rows



In [ ]:
df_filtered = df_without_ratio.filter(df_without_ratio.age >= 18)
df_filtered.show()


+------+---+------+----------+
|gender|age|income|profession|
+------+---+------+----------+
|  Male| 53|146783|   Teacher|
|  Male| 38|102661|    Doctor|
|  Male| 63| 27239|  Engineer|
|  Male| 49|133171|    Lawyer|
|Female| 33|144383|    Doctor|
|Female| 24| 95867|  Engineer|
|  Male| 52|112093|  Engineer|
|  Male| 32| 50041|    Doctor|
|Female| 45| 26404|    Lawyer|
|  Male| 55| 42273|    Lawyer|
|Female| 22| 37125|    Doctor|
|Female| 33|130449|    Lawyer|
|Female| 45| 29300|  Engineer|
|Female| 47| 76500|   Teacher|
|Female| 18|107528|   Teacher|
|Female| 46| 81486|    Doctor|
|  Male| 31| 76713|    Lawyer|
|  Male| 56| 95985|    Doctor|
|  Male| 41| 78577|  Engineer|
|  Male| 42| 23073|  Engineer|
+------+---+------+----------+
only showing top 20 rows



In [ ]:
from pyspark.sql.types import IntegerType, DoubleType

# Преобразование столбцов в числовой тип
df_filtered = df_filtered.withColumn("age", df_filtered["age"].cast(IntegerType()))
df_filtered = df_filtered.withColumn("income", df_filtered["income"].cast(DoubleType()))

# Проверьте схему данных после преобразования
df_filtered.printSchema()

from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Кодирование категориальных переменных Gender и Profession
gender_indexer = StringIndexer(inputCol="gender", outputCol="genderindex")
profession_indexer = StringIndexer(inputCol="profession", outputCol="professionindex")

# Применение индексирования
df_filtered_indexed = gender_indexer.fit(df_filtered).transform(df_filtered)
df_filtered_indexed = profession_indexer.fit(df_filtered_indexed).transform(df_filtered_indexed)

# Сбор всех признаков в вектор
assembler = VectorAssembler(inputCols=["age", "genderindex", "professionindex"], outputCol="features")
df_ml = assembler.transform(df_filtered_indexed).select("features", "income")

# Разделение на обучающую и тестовую выборки
train, test = df_ml.randomSplit([0.8, 0.2], seed=42)

# Обучение модели линейной регрессии
lr = LinearRegression(featuresCol="features", labelCol="income")
model = lr.fit(train)

# Прогнозирование на тестовых данных
predictions = model.transform(test)

# Оценка модели
evaluator = RegressionEvaluator(labelCol="income", predictionCol="prediction")
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

# Вывод метрик
print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}")


root
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- income: double (nullable = true)
 |-- profession: string (nullable = true)

RMSE: 37529.83, MAE: 32524.87, R2: -0.00
